In [1]:
import pandas as pd
import datapreprocessing

In [2]:
filepath ='../src/dataset/DataSet.csv'
dataset = pd.read_csv(filepath)

In [3]:
dataset.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,"<h3>We're Food52, and we've created a groundbr...","<p>Food52, a fast-growing, James Beard Award-w...",<ul>\r\n<li>Experience with content management...,NaN,f,t,f,Other,Internship,NaN,NaN,Marketing,f,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"<h3>90 Seconds, the worlds Cloud Video Product...",<p>Organised - Focused - Vibrant - Awesome!<br...,<p><b>What we expect from you:</b></p>\r\n<p>Y...,<h3><b>What you will get from us</b></h3>\r\n<...,f,t,f,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,f,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,<h3></h3>\r\n<p>Valor Services provides Workfo...,"<p>Our client, located in Houston, is actively...",<ul>\r\n<li>Implement pre-commissioning and co...,NaN,f,t,f,NaN,NaN,NaN,NaN,NaN,f,f
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,<p>Our passion for improving quality of life t...,<p><b>THE COMPANY: ESRI – Environmental System...,<ul>\r\n<li>\r\n<b>EDUCATION: </b>Bachelor’s o...,<p>Our culture is anything but corporate—we ha...,f,t,f,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,f,f
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,<p>SpotSource Solutions LLC is a Global Human ...,<p><b>JOB TITLE:</b> Itemization Review Manage...,<p><b>QUALIFICATIONS:</b></p>\r\n<ul>\r\n<li>R...,<p>Full Benefits Offered</p>,f,t,t,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,f,f


In [4]:
dataset = datapreprocessing.datasetprocess(dataset)

In [5]:
dataset.head()

,fraudulent,text
0,0,market intern marketingwer food weve creat gro...
1,0,custom servic cloud video product success seco...
2,0,commiss machineri assist cmavalor servic provi...
3,0,account execut washington dc salesour passion ...
4,0,bill review managerspotsourc solut llc global ...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split



# 使用TF-IDF表示文本特征
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(dataset['text'])

# 使用SMOTETomek技术进行过采样和欠采样
smt = SMOTETomek(random_state=42)
X_resampled, y_resampled = smt.fit_resample(X_tfidf, dataset['fraudulent'])

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [14]:
print(X_resampled.shape)
print('\n')
print(y_resampled.shape)

(31218, 172113)


(31218,)


In [15]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy as np

# 定义朴素贝叶斯、支持向量机和随机森林三个基学习器
base_estimators = [
    ('nb', GaussianNB()),
    ('svm', SVC(kernel='linear', probability=True)),
    ('rf', RandomForestClassifier(n_estimators=100))
]

# 定义元学习器，这里我们使用逻辑回归作为元学习器
meta_estimator = LogisticRegression()

# 定义 stacking 模型
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=meta_estimator)

# 定义交叉验证策略
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# 初始化保存各个基学习器的预测结果
base_estimators_predictions = np.zeros((X_train.shape[0], len(base_estimators)))

# 在训练集上进行交叉验证，得到各个基学习器的预测结果
for train_index, valid_index in cv.split(X_train, y_train):
    for i, (name, estimator) in enumerate(base_estimators):
        # 克隆基学习器
        clone_estimator = clone(estimator)
        # 在交叉验证的训练集上训练基学习器
        clone_estimator.fit(X_train[train_index], y_train[train_index])
        # 在交叉验证的验证集上进行预测
        base_estimators_predictions[valid_index, i] = clone_estimator.predict(X_train[valid_index])

# 在整个训练集上再次训练基学习器
for i, (name, estimator) in enumerate(base_estimators):
    estimator.fit(X_train, y_train)

# 使用基学习器的预测结果作为元特征，训练元学习器
meta_features = base_estimators_predictions
meta_estimator.fit(meta_features, y_train)

# 使用 stacking 模型进行预测
stacking_pred = stacking_model.predict(X_test)

# 输出分类报告
print("Classification Report:")
print(classification_report(y_test, stacking_pred))

# 计算 ROC AUC 得分
roc_auc = roc_auc_score(y_test, stacking_pred[:, 1])

# 输出 ROC AUC
print("ROC AUC:", roc_auc)

KeyError: '[6, 17, 33, 34, 75, 83, 92, 96, 119, 121, 127, 144, 150, 167, 173, 188, 194, 196, 210, 214, 216, 219, 221, 231, 243, 247, 250, 261, 265, 266, 274, 278, 279, 280, 283, 288, 290, 291, 312, 315, 321, 323, 327, 330, 334, 341, 344, 346, 351, 382, 396, 397, 401, 408, 409, 423, 428, 430, 447, 450, 456, 457, 459, 462, 464, 474, 476, 480, 482, 485, 487, 491, 499, 501, 503, 505, 506, 512, 536, 538, 550, 573, 601, 607, 616, 624, 626, 640, 644, 649, 651, 656, 657, 664, 702, 705, 715, 718, 725, 729, 733, 736, 741, 742, 747, 752, 762, 772, 775, 790, 791, 802, 811, 820, 821, 836, 843, 866, 873, 879, 890, 895, 896, 900, 905, 911, 912, 927, 928, 943, 959, 963, 965, 980, 985, 993, 996, 1012, 1023, 1026, 1033, 1035, 1043, 1045, 1047, 1050, 1052, 1053, 1056, 1075, 1078, 1087, 1092, 1103, 1106, 1108, 1114, 1121, 1133, 1147, 1163, 1179, 1182, 1183, 1188, 1190, 1197, 1201, 1202, 1215, 1227, 1243, 1246, 1247, 1266, 1276, 1280, 1288, 1290, 1298, 1302, 1303, 1311, 1318, 1322, 1323, 1328, 1330, 1334, 